In [1]:
import numpy as np
import cvxpy as cp

print(cp.installed_solvers())
assert "SCIPY" in cp.installed_solvers()
assert "GUROBI" in cp.installed_solvers()

['CLARABEL', 'GUROBI', 'OSQP', 'SCIPY', 'SCS']


References:
- [This](https://github.com/elena-ecn/benders-decomposition/blob/main/benders.py) Github repo was a huge help
- TUTORIAL BENDERS DECOMPOSITION IN RESTRUCTURED POWER SYSTEMS

\begin{split}
\min z = \mathbf{c}^T \mathbf{x} +\mathbf{d}^T \mathbf{y} \\
\textrm{s.t} \\
\mathbf{A} \mathbf{y} \geq \mathbf{b} \\
\mathbf{E} \mathbf{x} + \mathbf{F} \mathbf{y} \geq \mathbf{h} \\
\mathbf{x} \geq 0, \mathbf{y} \in \mathbf{S}
\end{split}

The main problem is defined as:
\begin{split}
\min \mathbf{d}^T \mathbf{y} \\
\textrm{s.t.} \\
\mathbf{A} \mathbf{y} \geq \mathbf{b} \\
\mathbf{y} \in S
\end{split}

The subproblem (primal) is defined as:
\begin{split}
\min \mathbf{c}^T \mathbf{x} \\
\textrm{s.t.} \\
\mathbf{E}\mathbf{x} \geq \mathbf{h} - \mathbf{F}\hat{\mathbf{y}}\\
\mathbf{x} \geq 0
\end{split}

The subproblem (dual) is defined as:
\begin{split}
\max (\mathbf{h} - \mathbf{F}\hat{\mathbf{y}} )^T \mathbf{u}\\
\textrm{s.t.}\\
\mathbf{E}^T \mathbf{u} \leq \mathbf{c}\\
\mathbf{u} \geq \mathbf{0}
\end{split}

The feasibility check subproblem is defined as:
\begin{split}
\min \mathbf{1}^T \mathbf{s} \\
\textrm{s.t.} \\
\mathbf{E}\mathbf{x} + \mathbf{I}\mathbf{s} \geq \mathbf{h} - \mathbf{F}\hat{\mathbf{y}} \rightarrow \mathbf{u}^r\\
\mathbf{x}\geq 0, \space \mathbf{s} \geq 0\\
\end{split}

The algorithm is:

0) Solve Main Problem $\rightarrow$ $\hat{\mathbf{y}}$
    - Lower bound on solution is establised at $\mathbf{d}^T \hat{\mathbf{y}}$
1) Solve Sub Problem $\rightarrow$ $\hat{\mathbf{x}}$ or $\hat{\mathbf{u}}$
    - If optimal solution is found:
        - Upper bound on solution at $\mathbf{d}^T \hat{\mathbf{y}} + \mathbf{c}^T\hat{\mathbf{x}}$ or $\mathbf{d}^T \hat{\mathbf{y}} + (\mathbf{h}-\mathbf{F}\hat{\mathbf{y}})^T\hat{\mathbf{u}}$
        - If the upper and lower bounds are close enough to a tolerance, stop solution loop and extract variable values.
            - Else, add optimality constraint to Main Problem of form of $n \geq (\mathbf{h} - \mathbf{F}\mathbf{y})^T \hat{\mathbf{u}}^p$
    - Else, if dual is unbounded (primal is infeasible):
        - Solve the feasibility check subproblem
        - Add feasibility constraint of form $0 \geq (\mathbf{h}-\mathbf{F}\mathbf{y})^T \hat{\mathbf{u}}^r$
2) Solve Main Problem
    - This is a modified form of the original main problem. Include all the optimality and feasibility constraints, and make the objective function $\mathbf{d}^T\mathbf{y} + n$, where $n$ is a slack variable of sorts to allow for minimizing the lower bound $z_{lower}$ subject to $z_{lower} \geq \mathbf{d}^T\mathbf{y}$

# Example 4.1

\begin{split}
\min x + y \\
\textrm{s.t.} 2x + y \geq 3 \\
x \geq 0 \\ 
y \in \{-5,-4,\dots,3,4\} \\
\end{split}

In [2]:
# Example 4.1
c = np.array([1.])
d = np.array([1.])
E = np.array([2.])
F = np.array([1.])
h = np.array([3.])

In [3]:
# Main Problem, Itr 1
z_lower = cp.Variable((1,1))
y = cp.Variable((1,1), integer=True)
constraints = [y >= -5., y <= 4., z_lower >= y]
objective = z_lower
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="GUROBI")
z_lower_1 = prob.value
y_hat_1 = y.value
(y_hat_1, z_lower_1, prob.value, prob.status)

Restricted license - for non-production use only - expires 2026-11-23


(array([[-5.]]), -5.0, -5.0, 'optimal')

In [4]:
# Primal Subproblem, Itr 1
x = cp.Variable((1,1))
constraints = [x >= 0, 2*x + y_hat_1 >= 3]
objective = x
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve()
x_hat_1 = x.value
z_upper_1 = d @ y_hat_1 + c @ x_hat_1
(x_hat_1, z_upper_1, prob.value, prob.status)

(array([[4.]]), array([-1.]), 3.9999999996766573, 'optimal')

In [5]:
# Dual Subproblem, Itr 1
u = cp.Variable((1,1))
constraints = [u >= 0, E@u <= c]
objective = (h - F @ y_hat_1) @ u
prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve()
u_hat_1 = u.value
z_upper_1 = d @ y_hat_1 + (h - F @ y_hat_1) @ u_hat_1
(u_hat_1, constraints[1].dual_value, z_upper_1, prob.value, prob.status)

(array([[0.5]]), array([4.]), array([-1.]), 3.999999999486556, 'optimal')

In [6]:
assert z_upper_1 > z_lower_1

In [7]:
# Main Problem, Itr 2
z_lower = cp.Variable((1,1))
y = cp.Variable((1,1), integer=True)
n = cp.Variable((1,1))
constraints = [y >= -5., y <= 4., z_lower >= d @ y + (h - F @ y) @ u_hat_1]
objective = z_lower
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="GUROBI", reoptimize=True)
z_lower_2 = prob.value
y_hat_2 = y.value
(y.value, z_lower_2, prob.value, prob.status)

(array([[-5.]]), -1.000000000513444, -1.000000000513444, 'optimal')

In [8]:
# Main Problem, Itr 2
# This also works. 
# The IIT tutorial paper uses z_lower >= d @ y + (h - F @ y) @ u_hat_1
# https://github.com/elena-ecn/benders-decomposition/blob/main/benders.py uses (h - F @ y) @ u_hat_1
# It seems that n = z - d @ y, so its a sort of slack variable
z_lower = cp.Variable((1,1))
y = cp.Variable((1,1), integer=True)
n = cp.Variable((1,1))
constraints = [y >= -5., y <= 4., n >= (h - F@y) @ u_hat_1]
objective = y + n
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="GUROBI", reoptimize=False)
z_lower_2 = prob.value
y_hat_2 = y.value
(y.value, n.value, z_lower_2, prob.value, prob.status)

(array([[-5.]]),
 array([[4.]]),
 -1.000000000513444,
 -1.000000000513444,
 'optimal')

# Example 5.1

\begin{split}
\min x_1 + 3x_2 + y_1 + 4y_2 \\
\textrm{s.t.} \\
-2x_1 - x_2 + y_1 - 2y_2 \geq 1 \\
2x_1 + 2x_2 - y_1 + 3y_2 \geq 1 \\
x_1, x_2 \geq 0, \space y_1, y_2 \geq 0
\end{split}

In [9]:
c = np.array([1., 3.])
d = np.array([1., 4.])
E = np.array([[-2., -1.], [2., 2.]])
F = np.array([[1., -2.], [-1., 3.]])
h = np.array([1., 1.])

In [10]:
# Main Problem (1)
z_lower = cp.Variable(2)
y = cp.Variable(2)
constraints = [y >= np.zeros_like(0)]
objective = d @ y
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="GUROBI")
z_lower_1 = d @ y.value  # no slack var in first main iteration, so...whatevs
y_hat_1 = y.value
y_hat_1, z_lower_1, prob.status

(array([0., 0.]), np.float64(0.0), 'optimal')

In [11]:
# Primal Subproblem, Itr 1
x = cp.Variable(2)
constraints = [x >= np.zeros_like(x), E @ x >= h - F @ y_hat_1]
objective = c @ x
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve()
x_hat_1 = x.value
print(x_hat_1)
#z_upper_1 = d @ y_hat_1 + c @ x_hat_1
(x_hat_1, z_upper_1, prob.value, prob.status)

None


(None, array([-1.]), inf, 'infeasible')

In [12]:
# Dual Subproblem, Itr 1
u = cp.Variable((2,1))
constraints = [
    u >= np.zeros_like(u), 
    E.T @ u <= c
]
objective = (h - F @ y_hat_1) @ u
prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve()
print(prob.status)
#u_hat_1 = u.value
#z_upper_1 = d @ y_hat_1 + (h - F @ y_hat_1) @ u_hat_1
#(u_hat_1, z_upper_1, prob.value, prob.status)

unbounded


In [13]:
# The primal subproblme is infeasible and dual subproblem is unbounded
# This requires the feasibility check subproblem to generate a feasibility cut
s = cp.Variable(2)
x = cp.Variable(2)
objective = np.array([1., 1.]) @ s
constraints = [x >= 0, s>=0, E@x + np.eye(2)@s >= h - F @ y_hat_1]
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="GUROBI")
print(prob.status)
# The dual values of the constraints (Lagrange multipliers) are stored each constraint
# We are interested in the duals of the decision variables, not their feasibility
# To get this by hand, you can construct the Lagrangian using the KKT conditions
r_hat_1 = constraints[2].dual_value
print(r_hat_1, s.value.tolist(), x.value.tolist(), prob.value)

optimal
[1.  0.5] [1.5, 0.0] [0.0, 0.5] 1.5


In [14]:
# Main Problem (2)
# we add a feasibility cut: (h-Fy)^T @ u <= 0
# The alternate form would be {solution to feasibility check} - (y - y_hat)^T @ F^T @ u <= 0
# where "u" is the dual values from the feasiblity check
z_lower = cp.Variable(2)
y = cp.Variable(2, integer=True)
constraints = [y >= 0, (h-F@y)@r_hat_1 <= 0 ]
objective = d @ y
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="GUROBI")
z_lower_2 = d @ y.value
y_hat_2 = y.value
y_hat_2, z_lower_2, prob.status

(array([ 3., -0.]), np.float64(3.0), 'optimal')

In [15]:
# Primal Subproblem, Itr 2
x = cp.Variable(2)
constraints = [x >= np.zeros_like(x), E @ x >= h - F @ y_hat_2]
objective = c @ x
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="SCIPY")
x_hat_2 = x.value
print(x_hat_2)
z_upper_2 = d @ y_hat_2 + c @ x_hat_2
(x_hat_2, z_upper_2, prob.value, prob.status)

[-0.  2.]


(array([-0.,  2.]), np.float64(9.0), np.float64(6.0), 'optimal')

In [16]:
# Dual Subproblem, Itr 2
u = cp.Variable(2)
constraints = [
    u >= np.zeros_like(u), 
    E.T @ u <= c
]
objective = (h - F @ y_hat_2) @ u
prob = cp.Problem(cp.Maximize(objective), constraints)
prob.solve(solver="SCIPY")  # only SCIPY or CUROBI return correct answers here. what the fuck? SCS gives 19!!!
print(prob.status)
u_hat_2 = u.value
z_upper_2 = d @ y_hat_2 + (h - F @ y_hat_2) @ u_hat_2
(u_hat_2, z_upper_2, prob.value, prob.status)

optimal


(array([2. , 2.5]), np.float64(9.0), np.float64(6.0), 'optimal')

In [17]:
# Main Problem (3)
# here, the objective function takes the form d@y + n
# the "n" was always there before it just didn't do anything
# see iteration 2 of example 4.1 -> its a sort of slack variable n = z - d @ y 
# when the objective function is Z
# iteration 2 subproblem was feasible, so we add a optimality constraint of form n >= (h - Fy)^T u_hat_2
z_lower = cp.Variable(2)
y = cp.Variable(2)
n = cp.Variable(1)
constraints = [y >= np.zeros_like(0), (h-F@y)@r_hat_1 <= 0, n >= (h - F@y) @ u_hat_2]
objective = d @ y + n
prob = cp.Problem(cp.Minimize(objective), constraints)
prob.solve(solver="GUROBI")
z_lower_3 = d @ y.value + n.value
y_hat_3 = y.value
y_hat_3, z_lower_3, prob.status

(array([3., 0.]), array([9.]), 'optimal')

In [18]:
assert np.isclose(z_lower_3, z_upper_2)